# Getting started with the Data API

### **Let's search & download some imagery of farmland near Stockton, CA. Here are the steps we'll follow:**

1. Define an Area of Interest (AOI)
2. Save our AOI's coordinates to GeoJSON format
3. Create a few search filters
4. Search for imagery using those filters
5. Activate an image for downloading
6. Download an image

### Requirements
- Python 2.7 or 3+
- requests
- A [Planet API Key](https://www.planet.com/account/#/)

## Set Up API Key

In [2]:
import config

api_key = config.API_KEY


## Define an Area of Interest

An **Area of Interest** (or *AOI*) is how we define the geographic "window" out of which we want to get data.

For the Data API, this could be a simple bounding box with four corners, or a more complex shape, as long as the definition is in [GeoJSON](http://geojson.org/) format. 

For this example, let's just use a simple box. To make it easy, I'll use [geojson.io](http://geojson.io/) to quickly draw a shape & generate GeoJSON output for our box:

![geojsonio.png](images/geojsonio.png)

We only need the "geometry" object for our Data API request:

In [3]:
# Stockton,  CA bounding box (created via geojson.io) 
geojson_geometry = {
  "type": "Polygon",
  "coordinates": [
    [ 
        [119.56033, 32.79919],
        [117.36145, 32.81461],
        [119.55682, 34.77144],
        [119.55682, 34.77144],
    ]
  ]
}

In [6]:
import os
import requests

item_id = "20250109_235323_82_2417"
item_type = "PSScene"
asset_type = "basic_beta_udm"

# setup auth
session = requests.Session()
# session.auth = (os.environ['PLANET_API_KEY'], '')
session.auth = (config.API_KEY, '')


# request an item
item = \
  session.get(
    ("https://api.planet.com/data/v1/item-types/" +
    "{}/items/{}/assets/").format(item_type, item_id))
print(item.json())
# extract the activation url from the item for the desired asset
item_activation_url = item.json()[asset_type]["_links"]["activate"]

# request activation
response = session.post(item_activation_url)

print(response.status_code)

{}


KeyError: 'basic_beta_udm'

## Create Filters

Now let's set up some **filters** to further constrain our Data API search:

In [14]:
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2025-01-09T00:00:00.000Z",
    "lte": "2025-01-10T00:00:00.000Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

## Searching: Items and Assets

Planet's products are categorized as **items** and **assets**: an item is a single picture taken by a satellite at a certain time. Items have multiple asset types including the image in different formats, along with supporting metadata files.

For this demonstration, let's get a satellite image that is best suited for analytic applications; i.e., a 4-band image with spectral data for Red, Green, Blue and Near-infrared values. To get the image we want, we will specify an item type of `PSScene`, and asset type `ps4b_analytic` (to get a PSScene4Band Analytic asset).

You can learn more about item & asset types in Planet's Data API [here](https://developers.planet.com/docs/apis/data/items-assets/).

Now let's search for all the items that match our filters:

In [22]:
import json
import requests
from requests.auth import HTTPBasicAuth

item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": date_range_filter#combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(API_KEY, ''),
    json=search_request)

geojson = search_result.json()

print(geojson)
# let's look at the first result
print(list(geojson.items())[1][1][0])

{'_links': {'_first': 'https://api.planet.com/data/v1/searches/e06268668cfe49059fcbb550ef051d9e/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D', '_next': 'https://api.planet.com/data/v1/searches/e06268668cfe49059fcbb550ef051d9e/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6ICIyMDI1LTAxLTEwVDEwOjQyOjE3LjAwMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMjUwMTA5XzExMjkyNF8xNl8yNGRjIiwgInNvcnRfcHJldiI6IGZhbHNlLCAicXVlcnlfcGFyYW1zIjoge319', '_self': 'https://api.planet.com/data/v1/searches/e06268668cfe49059fcbb550ef051d9e/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D'}, 'features': [{'_links': {'_self': 'https://api

Our search returns metadata for all of the images within our AOI that match our date range and cloud coverage filters. It looks like there are multiple images here; let's extract a list of just those image IDs:

In [16]:
# extract image IDs only
image_ids = [feature['id'] for feature in geojson['features']]
print(image_ids)

['20250109_235323_82_2417', '20250109_235325_66_2417', '20250109_235232_38_24cc', '20250109_235238_26_24cc', '20250109_235157_14_24cc', '20250109_235331_12_24cc', '20250109_235234_34_24cc', '20250109_235236_30_24cc', '20250109_235329_16_24cc', '20250109_235437_68_24cc', '20250109_235327_20_24cc', '20250109_235336_99_24cc', '20250109_235445_51_24cc', '20250109_235449_43_24cc', '20250109_045320_22_2516', '20250109_235404_40_24cc', '20250109_235439_64_24cc', '20250109_235333_07_24cc', '20250109_235301_75_24cc', '20250109_235342_86_24cc', '20250109_235718_03_24cc', '20250109_235719_99_24cc', '20250109_235427_89_24cc', '20250109_235721_95_24cc', '20250109_235317_41_24cc', '20250109_235309_58_24cc', '20250109_235315_45_24cc', '20250109_235305_66_24cc', '20250109_235358_52_24cc', '20250109_235352_65_24cc', '20250109_235408_31_24cc', '20250109_235246_09_24cc', '20250109_235240_21_24cc', '20250109_235325_24_24cc', '20250109_235414_19_24cc', '20250109_235228_47_24cc', '20250109_235307_62_24cc', 

Since we just want a single image, and this is only a demonstration, for our purposes here we can arbitrarily select the first image in that list. Let's do that, and get the `asset` list available for that image:

In [20]:
# For demo purposes, just grab the first image ID
id0 = image_ids[1]
id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)

# Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
result = \
  requests.get(
    id0_url,
    auth=HTTPBasicAuth(API_KEY, '')
  )

print(result)
# List of asset types available for this particular satellite image
print(result.json().keys())


<Response [200]>
dict_keys([])


 ## Activation and Downloading
 
The Data API does not pre-generate assets, so they are not always immediately available to download. In order to download an asset, we first have to **activate** it.

Please note that while this notebook's workflow is fast, it involves accessing a download link for full Planet scenes. Accessing this link will charge you for downloading the entire scene even if you only actually download metadata or a small clipped portion of the asset. Due to the quota model in place for the Data API, this is not as cost-effective as other methods. If you are working with a large amount of data and are concerned about quota, do not use this notebook and instead consider the [Orders API Clipping Tool](https://developers.planet.com/apis/orders/tools/#clip).

Remember, earlier we decided we wanted a color-corrected image best suited for *analytic* applications. We can check the status of the PSScene 4-Band analytic asset we want to download like so:
 

In [18]:
# This is "inactive" if the "ortho_analytic_4b" asset has not yet been activated; otherwise 'active'
print(result.json()['ortho_analytic_4b']['status'])

KeyError: 'ortho_analytic_4b'

Let's now go ahead and **activate** that asset for download:

In [ ]:
# Parse out useful links
links = result.json()[u"ortho_analytic_4b"]["_links"]
self_link = links["_self"]
activation_link = links["activate"]

# Request activation of the 'ortho_analytic_4b' asset:
activate_result = \
  requests.get(
    activation_link,
    auth=HTTPBasicAuth(API_KEY, '')
  )

At this point, we wait for the activation status for the asset we are requesting to change from `inactive` to `active`. We can monitor this by polling the "status" of the asset:

In [ ]:
activation_status_result = \
  requests.get(
    self_link,
    auth=HTTPBasicAuth(API_KEY, '')
  )
    
print(activation_status_result.json()["status"])

Once the asset has finished activating (status is "active"), we can download it. 

*Note: the download link on an active asset is temporary*

In [ ]:
# Image can be downloaded by making a GET with your Planet API key, from here:
download_link = activation_status_result.json()["location"]
print(download_link)

![stockton_thumb.png](images/stockton_thumb.png)